In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
load_dotenv()

loader = PyPDFDirectoryLoader('data/')
documents = loader.load()

In [5]:
print(documents)
# print(documents[0].page_content)

[Document(metadata={'source': 'data\\Student Council 2023-24-Roles and responsibilities.pdf', 'page': 0}, page_content='Student  Council  2023-24 \n \nWhat’s  Student  Council?  \n \nA Student  Council  is a representative  structure  through  which  students  in the college  can become  \ninvolved  in the  affairs of  the college,  working in  partnership with  the management, staff  and \nstudents for the benefit of the college and its students.  \nWhy have a Student Council?  \nThe Student Council is an increasingly common feature in many colleges and has worked to the  \nbenefit of the institutes. Students have a valuable contribution to make to the betterment  of their  \ncollege and their involvement in the operation of the college is itself a valuable part of the  \neducation process for the students. A Student Council provides an opportunity for students to  \nengage in a structured partnership with teachers, students and management in the operation of  \ntheir college. Student

In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
chunks = text_splitter.split_documents(documents)
chunks
# chunks[7]


[Document(metadata={'source': 'data\\Student Council 2023-24-Roles and responsibilities.pdf', 'page': 0}, page_content='Student  Council  2023-24 \n \nWhat’s  Student  Council?  \n \nA Student  Council  is a representative  structure  through  which  students  in the college  can become  \ninvolved  in the  affairs of  the college,  working in  partnership with  the management, staff  and \nstudents for the benefit of the college and its students.  \nWhy have a Student Council?  \nThe Student Council is an increasingly common feature in many colleges and has worked to the  \nbenefit of the institutes. Students have a valuable contribution to make to the betterment  of their  \ncollege and their involvement in the operation of the college is itself a valuable part of the  \neducation process for the students. A Student Council provides an opportunity for students to  \nengage in a structured partnership with teachers, students and management in the operation of  \ntheir college. Student

In [8]:
from langchain_community.embeddings import OllamaEmbeddings

from langchain_community.vectorstores import Chroma
# from langchain_community.llms import Ollama

# llm = Ollama(model="llama3")

index = Chroma.from_documents(chunks, OllamaEmbeddings(model="nomic-embed-text:latest ",show_progress=True))


OllamaEmbeddings: 100%|██████████| 5/5 [00:25<00:00,  5.06s/it]


In [9]:
retriever = index.as_retriever()

In [10]:
retriever.get_relevant_documents("what is student council?")

C:\Users\Sachin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


[Document(metadata={'page': 0, 'source': 'data\\Student Council 2023-24-Roles and responsibilities.pdf'}, page_content='Student  Council  2023-24 \n \nWhat’s  Student  Council?  \n \nA Student  Council  is a representative  structure  through  which  students  in the college  can become  \ninvolved  in the  affairs of  the college,  working in  partnership with  the management, staff  and \nstudents for the benefit of the college and its students.  \nWhy have a Student Council?  \nThe Student Council is an increasingly common feature in many colleges and has worked to the  \nbenefit of the institutes. Students have a valuable contribution to make to the betterment  of their  \ncollege and their involvement in the operation of the college is itself a valuable part of the  \neducation process for the students. A Student Council provides an opportunity for students to  \nengage in a structured partnership with teachers, students and management in the operation of  \ntheir college. Student

In [11]:
from operator import itemgetter

template = """

Answer the following question:
Question: {question}

Answer the question based only on the following context:
{context}

"""

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

prompt = ChatPromptTemplate.from_template(template)
model = Ollama(model="llama3:latest")

In [16]:
rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | StrOutputParser()
)

In [17]:
rag_chain.invoke({"question": "what is student council?"})

OllamaEmbeddings:   0%|          | 0/1 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


'Student council is a group of students who are elected by their peers to represent the student body and work together to improve the university experience for all students. The student council typically consists of various positions, including President, Vice-President, General Secretary, Treasurer, and Wing Leaders.\n\nThe main responsibility of the student council is to listen to the concerns and ideas of students, and then work with other university departments and administrators to address those concerns and make positive changes. This can include organizing events, planning activities, and advocating for policy changes that benefit students.\n\nIn addition to these responsibilities, some student councils may also have specific goals or focus areas, such as promoting diversity and inclusion, supporting student mental health, or enhancing campus sustainability.\n\nOverall, the student council plays a crucial role in bringing together students from different backgrounds and discipli